### Building a baseline model

The goal is to build a baseline model that could be easily deployed to production and improved further on by measuring impact of each iteration relative to its baseline.

Definining config params. This needs to be moved to a proper config file.

In [3]:
# TODO: Move everything to a config file
train_data_path = "../data/train_data.parquet"
val_data_path = "../data/val_data.parquet"
test_data_path = "../data/test_data.parquet"

cat_features = ["PUZone", "DOZone"]
num_features = ["day_of_week", "hour_of_day"]
target_col = "target"

loss_function = "RMSE"
eval_metric = "RMSE"
verbose=0
task_type="GPU"

# Grid search params (aka hypothesis space)    
param_grid = {"iterations": [700, 800, 900],
              "learning_rate": [0.08, 0.1],
              "depth": [8, 9, 10]}

In [4]:
from catboost import CatBoostRegressor, Pool
from pandas import read_parquet

def train_regressor(train_data: Pool, 
                    eval_set: Pool = None, 
                    iterations: int = None, 
                    learning_rate: float = None, 
                    depth: int = None, 
                    loss_function: str = "RMSE", 
                    verbose: str = 1, 
                    eval_metric: str = "RMSE",
                    task_type: str = "CPU") -> CatBoostRegressor:
    
    model = CatBoostRegressor(iterations=iterations, 
                              learning_rate=learning_rate, 
                              depth=depth, 
                              loss_function=loss_function, 
                              verbose=verbose, 
                              eval_metric=eval_metric,
                              task_type=task_type)
    model.fit(X=train_data, 
              eval_set=eval_set)
    
    return model

In [3]:
train_df = read_parquet(path=train_data_path)
val_df = read_parquet(path=val_data_path)

train_data = Pool(data=train_df.loc[:, cat_features + num_features], 
                  label=train_df.loc[:, target_col], 
                  cat_features=cat_features)

val_data = Pool(data=val_df.loc[:, cat_features + num_features], 
                label=val_df.loc[:, target_col], 
                cat_features=cat_features)

Running hyperparameter tuning via grid search on validation set. For gradient-boosting, parameters are coupled, so we cannot set the parameters one after the other, thus grid search is applied. I chose deep trees (`depth` param) and a high number of trees (`iterations` param) because we have very few features in the training set and some of them contain a very high number of levels (e.g. pickup / dropoff columns contain a lot of categories, thus a lot of splits are needed to fit the patterns).

In [4]:
for iterations in param_grid.get("iterations"):
    for learning_rate in param_grid.get("learning_rate"):
        for depth in param_grid.get("depth"):
            model = train_regressor(train_data=train_data,
                                    eval_set=val_data,
                                    iterations=iterations,
                                    learning_rate=learning_rate,
                                    depth=depth,
                                    loss_function=loss_function,
                                    verbose=verbose,
                                    eval_metric=eval_metric,
                                    task_type=task_type)
            print(f"iterations={iterations}, learning_rate={learning_rate}, depth={depth} - {model.best_score_}")

iterations=700, learning_rate=0.08, depth=8 - {'learn': {'RMSE': 4.4394449423798195}, 'validation': {'RMSE': 4.367064699412806}}
iterations=700, learning_rate=0.08, depth=9 - {'learn': {'RMSE': 4.378100771932876}, 'validation': {'RMSE': 4.329276767751926}}
iterations=700, learning_rate=0.08, depth=10 - {'learn': {'RMSE': 4.318350863171308}, 'validation': {'RMSE': 4.293119452966029}}
iterations=700, learning_rate=0.1, depth=8 - {'learn': {'RMSE': 4.401306829604488}, 'validation': {'RMSE': 4.339787279239915}}
iterations=700, learning_rate=0.1, depth=9 - {'learn': {'RMSE': 4.336895018173773}, 'validation': {'RMSE': 4.2979590386132935}}
iterations=700, learning_rate=0.1, depth=10 - {'learn': {'RMSE': 4.269373692936791}, 'validation': {'RMSE': 4.2603080552205705}}
iterations=800, learning_rate=0.08, depth=8 - {'learn': {'RMSE': 4.418457899949425}, 'validation': {'RMSE': 4.352996769105687}}
iterations=800, learning_rate=0.08, depth=9 - {'learn': {'RMSE': 4.357617625903226}, 'validation': {'R

As we can see, the last parameter set with `iterations=900`, `learning_rate=0.1`, `depth=10` produced the best performing model. Though, a slight increase of number of trees as well as tree depth would probably improve these numbers a little bit, but this increase would be negligible and thus I will not invest more time into it for this task since training a high number of deep trees is a time consuming task.

In [7]:
from pandas import concat

train_val_df = concat(objs=[train_df, val_df], axis=0)

train_val_data = Pool(data=train_val_df.loc[:, cat_features + num_features],
                      label=train_val_df.loc[:, target_col],
                      cat_features=cat_features)

best_model = train_regressor(train_data=train_val_data,
                             iterations=900,
                             learning_rate=0.1,
                             depth=10,
                             loss_function=loss_function,
                             verbose=50,
                             eval_metric=eval_metric)

0:	learn: 11.1975823	total: 531ms	remaining: 7m 56s
50:	learn: 4.9091318	total: 26.2s	remaining: 7m 15s
100:	learn: 4.7541872	total: 55.3s	remaining: 7m 17s
150:	learn: 4.6557700	total: 1m 26s	remaining: 7m 8s
200:	learn: 4.5957180	total: 1m 56s	remaining: 6m 43s
250:	learn: 4.5545292	total: 2m 25s	remaining: 6m 15s
300:	learn: 4.5230206	total: 2m 53s	remaining: 5m 46s
350:	learn: 4.4950327	total: 3m 23s	remaining: 5m 18s
400:	learn: 4.4707273	total: 3m 54s	remaining: 4m 51s
450:	learn: 4.4477452	total: 4m 23s	remaining: 4m 22s
500:	learn: 4.4270635	total: 4m 54s	remaining: 3m 54s
550:	learn: 4.4121116	total: 5m 26s	remaining: 3m 26s
600:	learn: 4.3948765	total: 5m 57s	remaining: 2m 57s
650:	learn: 4.3823150	total: 6m 30s	remaining: 2m 29s
700:	learn: 4.3702172	total: 7m 3s	remaining: 2m
750:	learn: 4.3598468	total: 7m 34s	remaining: 1m 30s
800:	learn: 4.3501607	total: 8m 4s	remaining: 59.9s
850:	learn: 4.3414249	total: 8m 36s	remaining: 29.7s
899:	learn: 4.3321068	total: 9m 8s	remaini

In [10]:
best_model.save_model("../models/best_model.cb")

Calculating best model performance estimates on the test set. I will be using [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error) to measure the performance because it's much more intuitive than [RMSE](https://www.statisticshowto.com/probability-and-statistics/regression-analysis/rmse-root-mean-square-error/), thus it's easier to explain it to non-technical people. 

In [6]:
from catboost import CatBoostRegressor, Pool
from pandas import read_parquet

test_df = read_parquet(path=test_data_path)
test_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,PUBorough,PUZone,PUservice_zone,DOBorough,DOZone,DOservice_zone,target,trip_duration_in_minutes,day_of_week,hour_of_day
579,1,2019-12-24 23:56:07,2019-12-25 00:11:01,1.0,4.80,1.0,N,142,116,1,...,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Hamilton Heights,Boro Zone,21.30,14.900000,1,23
581,2,2019-12-25 00:10:41,2019-12-25 00:27:32,1.0,4.60,1.0,N,142,116,1,...,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Hamilton Heights,Boro Zone,19.80,16.850000,2,0
582,2,2019-12-25 01:33:11,2019-12-25 01:46:47,1.0,6.53,1.0,N,142,116,1,...,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Hamilton Heights,Boro Zone,24.30,13.600000,2,1
583,1,2019-12-25 01:44:38,2019-12-25 01:59:36,2.0,5.40,1.0,N,142,116,2,...,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Hamilton Heights,Boro Zone,22.30,14.966667,2,1
584,1,2019-12-25 09:39:42,2019-12-25 09:51:51,1.0,5.60,1.0,N,142,116,1,...,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Hamilton Heights,Boro Zone,20.80,12.150000,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896300,2,2019-12-30 20:30:02,2019-12-30 21:45:38,1.0,25.38,1.0,N,237,99,1,...,Manhattan,Upper East Side South,Yellow Zone,Staten Island,Freshkills Park,Boro Zone,95.54,75.600000,0,20
6896303,2,2019-12-26 23:58:51,2019-12-27 00:43:02,1.0,32.41,1.0,N,173,99,1,...,Queens,North Corona,Boro Zone,Staten Island,Freshkills Park,Boro Zone,97.54,44.183333,3,23
6896304,1,2019-12-27 08:44:39,2019-12-27 09:08:05,4.0,3.60,1.0,N,79,105,1,...,Manhattan,East Village,Yellow Zone,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone,20.30,23.433333,4,8
6896306,2,2019-12-26 17:13:40,2019-12-26 17:29:27,2.0,4.82,1.0,N,170,105,1,...,Manhattan,Murray Hill,Yellow Zone,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone,20.80,15.783333,3,17


In [10]:
best_model = CatBoostRegressor()
best_model.load_model(fname="../models/best_model.cb")

In [11]:
test_data = Pool(data=test_df.loc[:, cat_features + num_features],
                 label=test_df.loc[:, target_col],
                 cat_features=cat_features)

In [ ]:
best_model.eval_metrics(data=test_data, metrics=["RMSE", "MAPE"], ntree_start=899)

{'RMSE': [4.776290867192493], 'MAPE': [0.17570530426561248]}

This means that when we deploy this model into production, we expect it to undershoot / overshoot price estimation on average by 17.6 % across all prices ranges. There is clearly plenty of room for improvement :)

Now we retrain model with the best hyperparameter set and training + validation + test sets. This is going to be the final model that is ready to go to production.

In [24]:
from catboost import CatBoostRegressor, Pool
from pandas import read_parquet, concat

train_df = read_parquet(path=train_data_path)
val_df = read_parquet(path=val_data_path)

train_val_test_df = concat(objs=[train_df, val_df, test_df.loc[:, num_features + cat_features + [target_col]]], axis=0)

In [25]:
train_val_test_data = Pool(data=train_val_test_df.loc[:, cat_features + num_features],
                           label=train_val_test_df.loc[:, target_col],
                           cat_features=cat_features)

In [26]:
final_model = train_regressor(train_data=train_val_test_data,
                              iterations=900,
                              learning_rate=0.1,
                              depth=10,
                              loss_function=loss_function,
                              verbose=50,
                              eval_metric=eval_metric)

0:	learn: 11.2425057	total: 676ms	remaining: 10m 8s
50:	learn: 5.0145964	total: 31.7s	remaining: 8m 47s
100:	learn: 4.8410181	total: 1m 4s	remaining: 8m 33s
150:	learn: 4.7488201	total: 1m 42s	remaining: 8m 27s
200:	learn: 4.6860365	total: 2m 15s	remaining: 7m 52s
250:	learn: 4.6327558	total: 2m 55s	remaining: 7m 33s
300:	learn: 4.6003017	total: 3m 28s	remaining: 6m 55s
350:	learn: 4.5693923	total: 4m 5s	remaining: 6m 24s
400:	learn: 4.5461258	total: 4m 42s	remaining: 5m 51s
450:	learn: 4.5260135	total: 5m 20s	remaining: 5m 19s
500:	learn: 4.5100935	total: 5m 56s	remaining: 4m 43s
550:	learn: 4.4943110	total: 6m 30s	remaining: 4m 7s
600:	learn: 4.4777826	total: 7m 8s	remaining: 3m 33s
650:	learn: 4.4640345	total: 7m 46s	remaining: 2m 58s
700:	learn: 4.4520392	total: 8m 24s	remaining: 2m 23s
750:	learn: 4.4423004	total: 9m 1s	remaining: 1m 47s
800:	learn: 4.4314665	total: 9m 41s	remaining: 1m 11s
850:	learn: 4.4203450	total: 10m 20s	remaining: 35.7s
899:	learn: 4.4127931	total: 10m 57s	

In [27]:
final_model.save_model("../models/final_model.cb")

Now we're ready to deploy the model into production. In order to deploy the whole solution into production which consists of model training and model serving, we need:

- Rewrite the pipeline from both jupyter notebooks as a python package:
    - Create a config file with data urls, clipping parameters, features, hyperparameters, etc
    - Write dataclasses for processing config file and data
    - Write unit tests
    - Wrap this package into a Dockerfile and deploy it to a docker registry like [AWS ECR](https://aws.amazon.com/ecr/)
- Assuming new data arrives regularly and model needs to be updated on regular cadence - [Apache Airflow](https://airflow.apache.org/docs/) could be employed to orchestrate the pipeline by pulling aforesaid docker image from the registry with [KubernetesPodOperator](https://airflow.apache.org/docs/apache-airflow-providers-cncf-kubernetes/stable/operators.html#howto-operator-kubernetespodoperator). 
- Tools like [DVC](https://dvc.org/) and [Great Expectations](https://greatexpectations.io/) could help to version the datasets as well as ensure their quality.
- This whole homework assignment was written in [Pandas](https://pandas.pydata.org/docs/), but [Apache Spark](https://spark.apache.org/docs/latest/api/python/index.html) might be a better alternative once more features are added and it becomes too slow with Pandas.
- For feature management, orchestration and deployment, an open-source Feature Store [Feast](https://feast.dev/) would go a long way, ensuring that same features are easily discovered, re-used, correctly used in both, development and production.
- Deploy the final model and wrapping it into service with endpoint for predictions. [FastAPI](https://fastapi.tiangolo.com/) would probably be my go-to. [Flask] is another popular approach.
- Alternatively, we can create a data app with [Streamlit](https://docs.streamlit.io/). I have not looked into this one myself but its quite popular within the industry.